In [17]:
import pandas as pd
import pylab as plt
from tusp import string2ts, ts2string
from bot import prepareData,marketReturn
import numpy as np
%matplotlib inline
plt.rcParams['figure.figsize'] = 15, 8

# trayendo y preparando datos
df  = prepareData(pair="USDT_BTC",
                  start=string2ts("2017-07-01 00:00:00"),
                  end=string2ts("2018-03-01 00:00:00"),
                  period=3600*4)
"""
# calculando el retorno del mercado hasta el 4 quitil y del ultimo quintil
df_train = df.loc["2014-09-10 00:00:00":"2017-06-01 00:00:00",:]
df_test = df.loc["2017-06-01 00:00:00":"2017-09-01 00:00:00",:]
"""
per = 0.85
cum_return1 = marketReturn(df[:int(len(df)*per)]["close"])
 

df["ho"] = df["high"]-df["open"]
df["lo"] = df["low"]-df["open"]

df["outcome"] = (df["close"]>df["open"])*2-1

ts_1 = df.shift(1)
ts_2 = df.shift(2)

df["feat1"] = df["close"] > ts_1["close"]
df["feat2"] = df["close"] > ts_2["close"]
df.head()

,close,high,low,open,quoteVolume,volume,weightedAverage,volatility,ho,lo,outcome,feat1,feat2
date,,,,,,,,,,,,,
2017-07-01 00:00:00,2372.00001,2436.000000,2357.601825,2423.661388,2356.78160059,5.628487e+06,2388.209182,True,12.338612,-66.059562,-1,False,False
2017-07-01 04:00:00,2432.20700,2436.000000,2365.000000,2372.000010,1294.59081254,3.110462e+06,2402.660440,True,63.999990,-7.000010,1,True,False
2017-07-01 08:00:00,2423.00000,2442.000000,2390.000000,2432.207000,583.7512332,1.415255e+06,2424.415037,False,9.793000,-42.207000,-1,False,True
2017-07-01 12:00:00,2404.04346,2430.000000,2362.893580,2423.000000,1477.87532307,3.531118e+06,2389.320572,False,7.000000,-60.106420,-1,False,False
2017-07-01 16:00:00,2337.55000,2404.053533,2290.000000,2404.043460,2831.58218753,6.648026e+06,2347.813013,True,0.010072,-114.043460,-1,False,False


In [15]:
from stockstats import StockDataFrame
from profit import profit
from mldata import ml_data
from orders import orders

stock = StockDataFrame.retype(df.copy())



df["ml_data"] = ml_data(df["close"],la=10,pl=False)

w4 = pd.DataFrame(data={"w":df["ml_data"].values,"price":df["close"]})
w4["orders"] = orders(w4["w"])
w4["w"] = w4["w"].shift(1)
relReturn, vecReturn4 = profit(w4)
print "Retorno de estrategia ideal: %s"%(relReturn*100)

data = pd.DataFrame(data={"price":df["close"],"pricew":df["weightedAverage"],"volumen":df["volume"],
                          "rsi":stock["rsi_14"],"macd":stock["macd"],"macds":stock["macds"],"cci":stock["cci"],
                          "wr":stock["wr_14"],"ideal_w":w4["w"],"volatility": df["volatility"],"close":df["close"],"kdjk":stock["kdjk"],
                          "kdjd":stock["kdjd"],"kdjj":stock["kdjj"], "dma":stock["dma"], "trix":stock["trix"],
                          "vr":stock["vr"], "log_return":marketReturn(df["close"]), "feat1":df["feat1"], "feat2":df["feat2"]})


data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.fillna(method='bfill', inplace=True)

# separando datos para crear y evaluar el modelo de machine learning
train = data[1:int(len(data)*per)]
test = data[int(len(data)*per):]

features = ["feat1", "feat2"]

X, y = data[features], data["ideal_w"]
X_train, y_train, X_test, y_test = train[features], train["ideal_w"], test[features], test["ideal_w"]


Retorno de estrategia ideal: 28343.8759805


In [34]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

clf = XGBClassifier(n_estimators=9, learning_rate=0.75, gamma=12)

clf.fit(X_train, y_train)

pre_proba = clf.predict_proba(X_test)



In [18]:
def trade_with_stop(bar, slippage = 0, stop=None):
    """
    Given a bar, with a gain obtained by the closing price - opening price
    it applies a stop limit order to limit a negative loss
    If stop is equal to None, then it returns bar['gain']
    """
    bar['gain'] = bar['gain'] - slippage
    if stop<>None:
        real_stop = stop - slippage
        if bar['lo']<=stop:
            return real_stop
    # stop == None    
    return bar['gain']

In [37]:
STOP = -100
threshold = 0.5

w = pd.DataFrame(data={"w":pre,"close":test["close"], "pre_proba":pre_proba})

df["pre"] = df["pre_proba"] > threshold

Exception: Data must be 1-dimensional